In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Generación del DF</h4>

In [ ]:
ruta_capa_municipio_cuenca = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\Interseccion_Municipio\_3_2_Cuencas_escala_1_25000_CAR_Intersect_Mpio"

df_municipio_cuenca = pd.DataFrame.spatial.from_featureclass(ruta_capa_municipio_cuenca)

<h4>Preparación de nueva llave principal para la capa Municipio (dificultad 11001, Bogotá Urbana-Rural)</h4>

In [ ]:
capa_municipio = r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\municipio'
df_municipio = pd.DataFrame.spatial.from_featureclass(capa_municipio)

tabla_direccion = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\tbl_direccion"
df_direccion = pd.DataFrame.spatial.from_table(tabla_direccion)

join_municipio_direccion = pd.merge(df_municipio, df_direccion, how='inner',left_on='codigo_direccion', right_on='codigo_direccion')

<h4>Unificación Tabla -relación_capa_municipio-</h4>

In [ ]:
# Agrupamiento, lo que en SQL es Group by
df_tabla_cuenca = df_municipio_cuenca.groupby(['Codigo','CODDANE','Direccion']).agg({'Codigo':'count', 'CODDANE':'count','Direccion':'count'})

# Se cambia el nombre de las columnas asociadas a los conteos (funcion -rename-)
df_tabla_cuenca.rename({'Codigo':'conteo_codigo', 'CODDANE': 'conteo_CODDANE','Direccion':'conteo_Direccion'}, axis = 1, inplace = True)

# # Se resetan los índices
df_tabla_cuenca = df_tabla_cuenca.reset_index()

# Se cambia el nombre de la columna -CODDANE-
df_tabla_cuenca.rename({'CODDANE': 'codigo_municipio_dane', 'Codigo':'codigo_cuenca'}, axis = 1, inplace = True)

#Cambio de tipo de letra a la columna -Direccion-
df_tabla_cuenca['Direccion'] = df_tabla_cuenca['Direccion'].str.capitalize()

df_tabla_cuenca = df_tabla_cuenca[['codigo_cuenca', 'codigo_municipio_dane', 'Direccion']]

# Join para traer OBJECTID de la capa Municipio como llave principal para la relación
# Join comparando dos campos
df_nuevo_codigo = pd.merge(df_tabla_cuenca, join_municipio_direccion, how='inner',left_on=['Direccion','codigo_municipio_dane'], right_on=['nombre_direccion','codigo_municipio_dane'])

# Generación del campo incremental de acuerdo al número de capas
RID = []

# Se suma uno por el header dentro del data frame
for id_rid in range(1, df_nuevo_codigo.shape[0]+1):
    RID.append(id_rid)
 
# Se utiliza el método .loc para incorporar la columna, que en este caso fue la lista que se creo previamente   
df_nuevo_codigo.loc[:,'RID'] = RID

# Reselección de campos
df_tabla_cuenca_estandarizada = df_nuevo_codigo.loc[:, ['codigo_cuenca','codigo_municipio','RID']]

# Se estandariza la tabla final
df_r_municipio_cuenca = df_tabla_cuenca_estandarizada[['codigo_cuenca','codigo_municipio','RID']]

# Se reemplaza el caracter - por _ en los códigos de las cuencas
df_r_municipio_cuenca.loc[:, 'codigo_cuenca'] = df_r_municipio_cuenca.loc[:,'codigo_cuenca'].str.replace('-', '_')
df_r_municipio_cuenca['codigo_municipio'] = df_r_municipio_cuenca.loc[:,['codigo_municipio']].astype('int')

<h4>Exportado a .csv de la capa que relaciona -cuenca- con -municipio-</h4>
<p>Este .csv debe será migrado posteriormente cuando se cree la relación M:N<p>

In [ ]:
nombre_csv_municipio_cuenca = 'r_municipio_cuenca.csv'
df_r_municipio_cuenca.to_csv(nombre_csv_municipio_cuenca, index = False, encoding = 'UTF-8')
print("------------------------------- Se exporta el .csv: -r_municipio_cuenca- -------------------------------")